In [1]:
from kafka import KafkaProducer
from json import dumps
from kafka import KafkaConsumer
from json import loads
from pyspark.sql import SparkSession

In [2]:
# import os
# kafka = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0"
# spark = "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0"

# os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {},{} pyspark-shell".format(kafka, spark))

In [3]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

22/10/23 16:08:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/23 16:08:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
# KAFKA_BROKER='kafka-1:9092'
# KAFKA_TOPIC='Product'
# kafkaMessages = spark \
#   .readStream \
#   .format("kafka") \
#   .option("kafka.bootstrap.servers", KAFKA_BROKER) \
#   .option("subscribe", KAFKA_TOPIC) \
#   .option("startingOffsets", "earliest") \
#   .load()

In [ ]:
message = kafkaMessages.selectExpr("CAST(value AS STRING)")

In [6]:
data = spark.read.parquet('hdfs://namenode:9000/tiki/Product')

22/10/23 16:08:39 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [7]:
data.count()

3260

In [ ]:
# fileStream = message.writeStream \
#                       .trigger(processingTime='60 seconds')\
#                       .queryName("Persist the processed data") \
#                       .outputMode("append") \
#                       .format("parquet") \
#                       .option("path", "hdfs://namenode:9000/tiki/Product/") \
#                       .option("checkpointLocation", "hdfs://namenode:9000/tiki/checkpoints") \
#                       .start() \
#                       .awaitTermination()